In [1]:
from model.AnoGAN import AnoGAN
from fastprogress import master_bar, progress_bar
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
import torch

In [16]:
dataset = CIFAR10(root='data/', download=True, transform=ToTensor())
test_dataset = CIFAR10(root='data/', train=False, transform=ToTensor())

label = 1
dataset.targets = (torch.Tensor(dataset.targets) == 1).type(torch.float)
test_dataset.targets = (torch.Tensor(test_dataset.targets) == 1).type(torch.float)

dataset = torch.utils.data.Subset(dataset, (dataset.targets == label).nonzero(as_tuple=True)[0])
test_dataset = torch.utils.data.Subset(test_dataset, range(1250))
batch_size = 250
train_loader = DataLoader(dataset, batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size, num_workers=2)

Files already downloaded and verified


In [3]:
anogan = AnoGAN()

In [ ]:
epochs = 70
mb = master_bar(range(epochs))
AUROC_converg = []
ACC_converg = []
for epoch in mb:
    for j,(image,y) in enumerate(progress_bar(train_loader, parent = mb)):
        auroc, acc, g_loss, d_loss, fake, real = anogan.train(image, y, verbose = True)
        mb.child.comment = "gen_loss: {} dis_loss: {} fake: {} real: {}".format(g_loss, d_loss, fake, real)
    
    batch_auroc = []
    batch_acc = []
    for j,(batch,y) in enumerate(progress_bar(test_loader, parent = mb)):  
        preds, auroc, acc, g_loss, d_loss, fake, real = anogan.predict(batch, y, verbose = True)
        batch_auroc.append(float(auroc.data))
        batch_acc.append(float(acc.data))
    AUROC_converg.append(sum(batch_auroc) / len(batch_auroc))
    ACC_converg.append(sum (batch_acc) / len (batch_acc))
    mb.main_bar.comment = "AUROC: {} ACC:{}".format(sum(batch_auroc) / len(batch_auroc), sum(batch_acc) / len(batch_acc))

In [ ]:
torch.save(anogan, "AnoGAN_model_70.pth")

In [40]:
def VotingClassifier(input1, input2, input3, threshold = 0.5):
    return torch.mode(torch.where(torch.concat((input1, input2, input3), dim = 1)>.5, 1.,0.)).values

In [41]:
def AverageClassifier(input1, input2, input3, threshold = 0.5):
    return torch.where(torch.mean(torch.concat((input1, input2, input3), dim = 1), dim =1)>.5, 1.,0.)

In [43]:
def LinearEnsemble(nn.Module):
    def __init__(self, anoGAN, secondModel):
        super(MyEnsemble, self).__init__()
        self.anoGAN = modelA
        self.secondModel = modelB
        self.classifier = nn.Linear(in_features, 1) #define accordingly
        self.relu = nn.ReLU()

    def forward(self, x1, x2):
        x1 = self.firstModel(x1)
        x2 = self.secondModel(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(self.relu(x))
        return x

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
        False,  True, False,  True, False,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True,  True,
         True,  True,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True, False,  True,  True, False,  True,  True,
         True,  True, False,  True,  True,  True, False,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 

In [27]:
input1 = torch.rand(250, 1)
input2 = torch.rand(250, 1)
input3 = torch.rand(250, 1)

In [31]:
torch.concat((input1, input2, input3), dim = 1)

tensor([[9.0576e-01, 2.8849e-01, 3.5934e-02],
        [6.7506e-01, 9.6755e-01, 5.8058e-01],
        [5.4287e-01, 1.9431e-01, 3.3509e-01],
        [9.6710e-01, 2.2857e-01, 5.5371e-01],
        [6.3194e-01, 7.1887e-02, 2.2726e-01],
        [2.1070e-01, 5.1562e-01, 4.0798e-01],
        [9.1812e-01, 6.5026e-01, 5.6295e-01],
        [8.0242e-01, 8.9276e-01, 9.8960e-01],
        [4.3564e-01, 7.0617e-01, 1.7625e-01],
        [4.2774e-01, 9.5965e-01, 8.9670e-01],
        [8.2265e-01, 4.5825e-01, 4.5979e-01],
        [1.3519e-01, 8.4356e-02, 3.5421e-01],
        [6.7209e-01, 7.1920e-01, 5.3041e-02],
        [5.1039e-01, 1.9337e-01, 4.0851e-01],
        [9.9155e-01, 3.0309e-01, 2.6476e-01],
        [8.4773e-01, 3.3153e-01, 3.0312e-01],
        [9.9549e-01, 7.4510e-01, 6.3578e-01],
        [9.1351e-01, 8.0068e-01, 4.8528e-02],
        [5.7205e-01, 3.1795e-02, 8.1037e-01],
        [6.7060e-01, 2.3355e-01, 3.6285e-01],
        [6.4875e-01, 7.0650e-02, 2.9128e-01],
        [4.0811e-01, 7.1547e-01, 6